# Final project

### PREPARATION PART BEGINS

This final project is done by **Herman Havva**

### PREPARATION PART ENDS

-----------------------------

Install libraries:


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [23]:
!pip uninstall altair -y

Found existing installation: altair 6.0.0
Uninstalling altair-6.0.0:
  Successfully uninstalled altair-6.0.0


## Cool project, awesome project
### The roadmap of the project 
1. Clean-up and arranging of the data
2. **Analysis**

#### Analysis layout (what it attempts to show)
- **Main objective:** to create a comprehensive analysis of the mid 19 century Poltava governorate population data, explore the following points of interest
    1. Which districts(повіти) appear to be more wealthier in comparison to others, as in Russian Empire merchanants(wealthy prople) could leave some **treasures**(скарби) for "good times". We want to know where to look for them. We will introduce the wealth index: 
    $$WealthIndex = \frac{Feudal Land}{Number Of Feudals} * \frac{Slaves(Owned Plebs)}{TotalPlebs}$$

    2. Also, by any chance, lets explore the ratio of men and women born and dead, may be we can find something interesting, lets find it per district. Apart from the fact that there were several cholera outbreaks, I could not find any other interesting catastrophes. 
     
    
    (possibility) on the 10 tab compare the christians number to jews number (bar chart, x-axis is cities, two bars for each city), in total 2 bar charts for each revision
    (possibility) we can make a demographic chart by the 13_відтворення_населення_роки
    (possibility) we could extract the total population from the 2_Народонаселення_1848р and make a demographhic chart for 2 years

#### Results and observations
- The `wealthiest` districts are 
- There must have happened something bad `1848`, 
- In `1846` there was an anomally of a lot of girls being born


#### Clean-up and arranging the data 
- renamed the dataframes (.csv) to have reasonable names
- extracted raw data into pandas dataframes for the future analysis

In [3]:
import pandas as pd
import numpy as np
import io
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import altair as alt

In [ ]:
# Arrange the data for the Objective1 '

df_social = pd.read_csv('./data/1_народонаселення_1850.csv', header=[0, 1], index_col=0)

# ffill for unscrewed first row
top_level = df_social.columns.get_level_values(0).to_series()
clean_top_row = top_level.replace(r'^Unnamed:.*', np.nan, regex=True).ffill()
df_social.columns = pd.MultiIndex.from_arrays([clean_top_row, df_social.columns.get_level_values(1)])


# # 1. we need elites 
# df_elites = df_social.xs('о.п.', level=1, axis=1).T

In [83]:
# Predefined categories 
plebs_categories = [
    'Крестьян государственных разн. наименований',
    'Крестьян государственных водвор. на собств. земл.',
    'Однодворческих крестьян',
    'Крестьян государственных на льготе',
    'Крестьян помещичьих']

slaves_categories = ['Крестьян помещичьих']

elites_categories = [
    'Дворян потомственных',
    'Дворян личных',
    'Купцов 1-й гильдии',
    'Купцов 2-й гильдии',
    'Купцов 3-й гильдии',
    'Купцов 1-й гильдии на льготе',
    'Почетных граждан потомственных',
    'Почетных граждан личных'
]


In [88]:

# 1. we need elites 
df_elites = df_social.xs('о.п.', level=1, axis=1).T

df_elites.index.name = 'Повіт'

df_elites['elites_total'] = df_elites[elites_categories].sum(axis=1)

final_elites = df_elites[['elites_total']]


print(final_elites.loc[ :,'elites_total' ].head())

Повіт
Гадяцький               833
Зіньківський           1984
Золотоніський          1756
Кобеляцький            1270
Костянтиноградський     880
Name: elites_total, dtype: int64


In [ ]:
# 2. we need total plebs = free plebs + propritary slaves (owned by feodals) 
df_plebs = df_social.xs('о.п.', level=1, axis=1).T

df_plebs.index.name = 'Повіт'

# aggregation
df_plebs['plebs_total'] = df_plebs[plebs_categories].sum(axis=1) 
df_plebs['slaves_total'] = df_plebs[slaves_categories].sum(axis=1)

df_plebs_final = df_plebs[['plebs_total', 'slaves_total']].copy()
print(df_plebs_final.head()) 


                     plebs_total  slaves_total
Повіт                                         
Гадяцький                  54375         38956
Зіньківський               24214         19714
Золотоніський              63530         55491
Кобеляцький                33491         31918
Костянтиноградський        63445         59700


In [ ]:
# we need the to get the land owned by feodals


2

In [9]:
# Arrange data for the Objective2
total_reproduction_df = pd.read_csv('./data/13_відтворення_населення_роки.csv', 
                 header=[0, 1], 
                 encoding='utf-8', 
                 sep=',')         

top_level = total_reproduction_df.columns.get_level_values(0).to_series()

clean_top_row = top_level.replace(r'^Unnamed:.*', np.nan, regex=True).ffill()
total_reproduction_df.columns = pd.MultiIndex.from_arrays([clean_top_row, total_reproduction_df.columns.get_level_values(1)])

years_column = total_reproduction_df.iloc[:, 0] 

# add year index
total_reproduction_df.index = years_column
total_reproduction_df.index.name = 'Рік'

# remove old year column
total_reproduction_df = total_reproduction_df.iloc[:, 1:]

total_reproduction_df.index = pd.to_numeric(total_reproduction_df.index.to_series(), errors='coerce')


if total_reproduction_df.index.dtype == 'float64':
    total_reproduction_df.index = total_reproduction_df.index.astype('Int64')

total_reproduction_df = total_reproduction_df[total_reproduction_df.index.notna()]

total_reproduction_df.columns.names = ['Повіт', 'Стать']

print(total_reproduction_df['Губернія загалом'].head()) 

Стать        ч        ж     о.с.
Рік                             
1835    2459.0    282.0   2741.0
1836   11565.0  10271.0  21836.0
1837   10940.0   9259.0  20199.0
1838    6975.0   5280.0  12255.0
1839   14550.0  11621.0  26171.0


Lets have a population surplus chart for the provided time period `1835-1850`

In [6]:
CHART_HEIGHT = 500  
CHART_WIDTH = 200  
CORNER_RADIUS = 2


df_subset = total_reproduction_df['Губернія загалом'][['ч', 'ж']].copy()
df_subset.index.name = 'Рік'
df_subset = df_subset.reset_index()


# Було: [Рік, ч, ж]
# Стало: [Рік, Sex, Count]
source = df_subset.melt(id_vars=['Рік'], 
                        value_vars=['ч', 'ж'], 
                        var_name='Sex', 
                        value_name='Count')

source['Count'] = source['Count'].fillna(0)


hover = alt.selection_point(
    fields=['Рік'], on='mouseover', nearest=True, empty='none', clear='mouseout'
)

base = alt.Chart(source).encode(
    y=alt.Y('Рік:O', axis=None),
    tooltip=['Рік', 'Sex', 'Count']
).properties(height=CHART_HEIGHT)


zero_line = alt.Chart(source).mark_rule(color='gray', opacity=1.0, strokeWidth=0.5).encode(
    x=alt.datum(0)
)

# left part (males) - bars
left_bars = base.transform_filter(
    alt.datum.Sex == 'ч'
).mark_bar(
    cornerRadius=CORNER_RADIUS
    ).encode(
    x=alt.X('Count:Q', title='Чоловіки (Приріст)', sort='descending'), # Reversed axis
    color=alt.condition(
        alt.datum.Count > 0,
        alt.value('#3498DB'), 
        alt.value('#154360') 
    ),
    opacity=alt.condition(hover, alt.value(1), alt.value(0.6))
)

left = alt.layer(zero_line, left_bars).properties(
    title='Чоловіки',
    width=300
).add_params(hover)


# right (females) - bars
right_bars = base.transform_filter(
    alt.datum.Sex == 'ж'
).mark_bar(
    cornerRadius=CORNER_RADIUS
).encode(
    x=alt.X('Count:Q', title='Жінки (Приріст)'),
    color=alt.condition(
        alt.datum.Count > 0,
        alt.value('#E74C3C'), 
        alt.value('#78281F')  
    ),
    opacity=alt.condition(hover, alt.value(1), alt.value(0.6))
    
)

right = alt.layer(zero_line, right_bars).properties(
    title='Жінки',
    width=300
).add_params(hover)


#  years part
middle = base.encode(
    y=alt.Y('Рік:O', axis=None),
    text=alt.Text('Рік'),
    color=alt.condition(hover, alt.value('black'), alt.value('gray'))
).mark_text().properties(width=50).add_params(hover)


# final
chart = alt.hconcat(
    left, middle, right,
    spacing=5
).resolve_scale(
    y='shared'
).configure_view(
    stroke=None
).configure_axis(
    grid=False
)

chart.display()

/Users/hermanhavva/Documents/Kse/DataManipulationEssentials/Stat163FinalProject/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3701: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  exec(code_obj, self.user_global_ns, self.user_ns)


alt.HConcatChart(...)

#### Intermediate analysis
- Seems like something bad happened in `1848`, it could be Cholera. In Europe that period is refered as **"Springtime of Nations"**

- We can clearly see that apart from the year `1846`, there on average **more boys born than girls**. This conforms to the average ratio of `105:95` (the boys are born more frequently). 

In [32]:
import geopandas as gpd
import altair as alt
import pandas as pd

# 1. Завантажуємо ваш новостворений файл
gdf = gpd.read_file('poltava_named_regions.geojson')

# 2. Підготовка центроїдів для підписів
# Altair потребує окремих колонок lat/lon для тексту
# (Попередження про CRS можна ігнорувати для простої візуалізації)
gdf['centroid_lon'] = gdf.geometry.centroid.x
gdf['centroid_lat'] = gdf.geometry.centroid.y

# Замінюємо NaN (де join не спрацював) на "Невідомо", щоб графік не впав
gdf['name'] = gdf['name'].fillna('Невідомо')

# 3. Шар полігонів (Мапа)
polygons = alt.Chart(gdf).mark_geoshape(
    stroke='white',
    strokeWidth=1
).encode(
    # Фарбуємо в різні кольори за назвою. 
    # Якщо 'Невідомо' - буде сірим (залежить від схеми, або можна задати умову)
    color=alt.Color('name:N', legend=None), 
    tooltip=['name', 'id'] # При наведенні покаже назву
).properties(
    width=600,
    height=500,
    title='Мапа Полтавської Губернії (Перевірка GeoJSON)'
)

# 4. Шар тексту (Назви повітів)
labels = alt.Chart(gdf).mark_text(
    color='black',
    fontSize=10,
    font='Arial'
).encode(
    longitude='centroid_lon:Q',
    latitude='centroid_lat:Q',
    text='name:N'
)

chart = alt.layer(polygons, labels).project(
    type='identity',
     reflectY=True 
)

chart.display()

/var/folders/vr/0mztxq6x39jfmjxrj1vxtb380000gn/T/ipykernel_57406/857765572.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid_lon'] = gdf.geometry.centroid.x
/var/folders/vr/0mztxq6x39jfmjxrj1vxtb380000gn/T/ipykernel_57406/857765572.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid_lat'] = gdf.geometry.centroid.y


alt.LayerChart(...)